In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

import json
import os
from datetime import datetime
from pprint import pprint

import database
import helper
import utils

In [3]:
LOG = utils.CustomLogger("CustomLogger", log_level= "info", display_loglevel= False, display_datetime= False)
PICKLE_LIB = utils.PickleLib(data_path="./data", logger= LOG)

# Data Search

In [13]:
client = database.connect()
db = client.JiraRepos

print(db.list_collection_names())
collection = db['Jira']

['Spring', 'Mojang', 'SecondLife', 'JiraEcosystem', 'Mindville', 'Sonatype', 'Sakai', 'Jira', 'Apache', 'MariaDB', 'Hyperledger', 'Qt', 'IntelDAOS', 'JFrog', 'MongoDB', 'RedHat']


In [7]:
# results = collection.aggregate([
#     {
#         "$match": {
#             "fields.description": {
#                 "$exists": True, 
#                 "$type": "string",
#                 "$regex": ".*During an in-app update.*"
#             }
#         }
#     },
#     {
#         "$sample": {
#             "size": 5
#         }
#     }
# ])


# list = []

# for document in results:
#     list.append(document)
#     print("Id: " + str(document['id']) + " Length: " + str(len(document['fields']['summary'])) + ": " + document['fields']['summary'])

Id: 1795802 Length: 49: Checksummed file size doesn't match during update
Id: 1795162 Length: 49: Checksummed file size doesn't match during update


#### Summary

In [ ]:
def getInRange():
    results = collection.aggregate([
        {
            "$match": {
                "fields.summary": {
                    "$exists": True, 
                    "$type": "string"
                },
                "$expr": {
                    "$and": [
                        {"$lt": [{"$strLenCP": "$fields.summary"}, 70]},
                        {"$gt": [{"$strLenCP": "$fields.summary"}, 39]}
                    ]
                }
            }
        },
        {
            "$sample": {
                "size": 5
            }
        }
    ])
    return results

def getShorter():
    results = collection.aggregate([
        {
            "$match": {
                "fields.summary": {
                    "$exists": True, 
                    "$type": "string"
                },
                "$expr": {
                    "$and": [
                        {"$lt": [{"$strLenCP": "$fields.summary"}, 39]}
                    ]
                }
            }
        },
        {
            "$sample": {
                "size": 5
            }
        }
    ])
    return results

def getLonger():
    results = collection.aggregate([
        {
            "$match": {
                "fields.summary": {
                    "$exists": True, 
                    "$type": "string"
                },
                "$expr": {
                    "$and": [
                        {"$gt": [{"$strLenCP": "$fields.summary"}, 70]}
                    ]
                }
            }
        },
        {
            "$sample": {
                "size": 5
            }
        }
    ])
    return results

In [ ]:
summary = []

results = getInRange()
# results = getShorter()
# results = getLonger()

for document in results:
    summary.append(document)
    print("Id: " + str(document['id']) + " Length: " + str(len(document['fields']['summary'])) + ": " + document['fields']['summary'])

#### Arbitrary Structure

In [14]:
def findKeysByCode(data, target_code):
    matching_keys = []
    for key, value in data.items():
        if 'code' in value and value['code'] == target_code:
            matching_keys.append(key)
    return matching_keys

In [15]:
with open('issueTypeMapping.json') as f:
    mappedIssueTypes = json.load(f)

In [16]:
mappedTypes = findKeysByCode(mappedIssueTypes['Jira'], 'Bug Report')
mappedTypes

['Bug', 'Public Security Vulnerability']

In [17]:
tickets = []
# results = collection.aggregate([
#     {
#         "$match": {
#             "fields.description": {
#                 "$exists": True, 
#                 "$type": "string",
#                 "$regex": "As a"
#             },
#             "fields.issuetype.name": {
#                 "$in": mappedTypes
#             }
#         }
#     },
#     {
#         "$sample": {
#             "size": 10
#         }
#     }
# ])
results = collection.aggregate([
    {
        "$match": {
            "fields.description": {
                "$exists": True, 
                "$type": "string"
            },
            "fields.issuetype.name": {
                "$in": mappedTypes
            }
        }
    },
    {
        "$sample": {
            "size": 10
        }
    }
])

for document in results:
    tickets.append(document)
    print("Id: " + str(document['id']) + " / Description: " + document['fields']['description'])

Id: 262258 / Description: {panel:bgColor=#e7f4fa}
  *NOTE:* This bug report is for *JIRA Server*. Using *JIRA Cloud*? [See the corresponding bug report|http://jira.atlassian.com/browse/JRACLOUD-31834].
  {panel}

I cannot define environment when I am defining custom trigger like this:
{code}
window.ATL_JQ_PAGE_PROPS = AJS.$.extend(window.ATL_JQ_PAGE_PROPS, {
    'a698db21' : {
        triggerFunction: function(showCollectorDialog) {
            // show issue collector on click on trigger element
            AJS.$(".feedback-trigger").click(function(e) {
                e.preventDefault();
                showCollectorDialog();
            });
        }
    }
});
{code}

It should really be resolved when trigger is clicked, not when it's rendered - too many races are possible.
Id: 1394299 / Description: Hi,

I am seeing all branches active/merged branches in “merged” and “active” options.
Id: 612424 / Description: At step 3, clicking on *Next*, I get the following "Unknown exception":



User Story structure: 
Redhat: 13279134, 13254138, 13282377, 13405745, 14248927(has only the story)

#### Update See use_case_3.ipynb

# Ticket Creation

(Apache, 13066997) -> Status is Open, coulb be intersting 
(Jira, 284325) -> could be a good example for the Description completness Prompt!!
(Qt, 187366) -> intersting for Bug Report STructre!

In [4]:
### Globals ###
LOG.reset()
JIRA = "IntelDAOS"
FOLDERNAME = "update"

In [5]:
evo_df = PICKLE_LIB.pickle_load("./jiraEvolutions/load_evolution_dataframe(jiras=[_"+JIRA+"_])", 'gzip')

[Start] 🥒 Loading data from Pickle: "./jiraEvolutions/load_evolution_dataframe(jiras=[_IntelDAOS_]).pgzip"


	 Data: 100%|###################################################| 62.5M/62.5M [00:00<00:00, 155MB/s]

[ End ] Duration: 00:00:00.5462


In [6]:
sample_id = "16799"
sample = evo_df[evo_df["issue_id"] == sample_id]
sample

jira issue_id  history_order             field  field_evo_order  \
109491  IntelDAOS    16799              0           Summary                0   
109492  IntelDAOS    16799              0       Description                0   
109493  IntelDAOS    16799              0            Labels                0   
109494  IntelDAOS    16799              0            Labels                0   
109495  IntelDAOS    16799              0         IssueType                0   
109496  IntelDAOS    16799              0           Project                0   
109497  IntelDAOS    16799              0       CreatedDate                0   
109498  IntelDAOS    16799              0      ResolvedDate                0   
109499  IntelDAOS    16799              0            Status                0   
109500  IntelDAOS    16799              0          Priority                0   
109501  IntelDAOS    16799              0              Rank                0   
109502  IntelDAOS    16799              0           Creator                0   
109503  IntelDAOS    16799              0          Reporter                0   
109504  IntelDAOS    16799              0          Assignee                0   
109505  IntelDAOS    16799              1       Description                1   
109506  IntelDAOS    16799              2       Description                2   
109507  IntelDAOS    16799              3       Description                3   
109508  IntelDAOS    16799              4        Components                0   
109509  IntelDAOS    16799              5            Labels                1   
109510  IntelDAOS    16799              6          Assignee                1   
109511  IntelDAOS    16799              7          Assignee                2   
109512  IntelDAOS    16799              8          Comments                0   
109513  IntelDAOS    16799              9          Comments                1   
109514  IntelDAOS    16799             10     VersionsFixed                0   
109515  IntelDAOS    16799             11  VersionsAffected                0   
109516  IntelDAOS    16799             12     VersionsFixed                1   
109517  IntelDAOS    16799             12     VersionsFixed                1   
109518  IntelDAOS    16799             13              Rank                1   
109519  IntelDAOS    16799             14          Comments                2   
109520  IntelDAOS    16799             15            Labels                2   
109521  IntelDAOS    16799             16     VersionsFixed                2   
109522  IntelDAOS    16799             16     VersionsFixed                2   
109523  IntelDAOS    16799             17            Labels                3   
109524  IntelDAOS    16799             18        IssueLinks                0   
109525  IntelDAOS    16799             19            Labels                4   
109526  IntelDAOS    16799             20            Labels                5   
109527  IntelDAOS    16799             20            Labels                5   
109528  IntelDAOS    16799             21          Comments                3   
109529  IntelDAOS    16799             22        Resolution                0   
109530  IntelDAOS    16799             22            Status                1   
109531  IntelDAOS    16799             23            Parent                0   

        field_evo_first  field_evo_last  \
109491             True            True   
109492             True           False   
109493             True           False   
109494             True           False   
109495             True            True   
109496             True            True   
109497             True            True   
109498             True            True   
109499             True           False   
109500             True            True   
109501             True           False   
109502             True            True   
109503             True            True   
109504             True           False   
109505 

In [22]:
evolutionStep = 0
ticket = helper.createTicket(sample, evolutionStep)
ticket

/Users/piet/Development/source/master-thesis/helper.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row.loc[:, 'Comments'] = [comment_history]


,Jira,IssueId,EvoId,Summary,Description,VersionsAffected,IssueType,Project,Components,CreatedDate,...,Priority,Creator,Reporter,Resolution,IssueLinks,Labels,VersionsFixed,Assignee,TimeSpent,Comments
0,Jira,1388024,0,Commit log bug,"After several attempts, I was able to make only one successful commit, rest all attempts failed due to some ""non-fast forward"" error . However bitbucket commit log shows all commits by user (even the failed ones) which is very confusing. Someone care to fix this bug??",None,Bug,Bitbucket Cloud,User - Legacy,2016-02-11T18:31:05.000+0000,...,Medium,Geoff,jangdmanish,Invalid,None,migrated,None,None,None,"Empty DataFrame Columns: [Author, Created, Comment] Index: []"


# Ticket PreProcessing

In [23]:
def preprocessTickets(ticket):
    
    ### Convert CreatedDate and ResolvedDate to datetime
    c_date = ticket['CreatedDate'].values[0]
    c_dt_obj = datetime.strptime(c_date, '%Y-%m-%dT%H:%M:%S.%f%z')
    ticket['CreatedDate'] = c_dt_obj.strftime('%Y-%m-%d %H:%M:%S')
    r_date = ticket['ResolvedDate'].values[0]
    r_dt_obj = datetime.strptime(r_date, '%Y-%m-%dT%H:%M:%S.%f%z')
    ticket['ResolvedDate'] = r_dt_obj.strftime('%Y-%m-%d %H:%M:%S')

    ### Convert IssueId to Int
    id = ticket['IssueId'].values[0]
    ticket['IssueId'] = int(id)

preprocessTickets(ticket)

In [24]:
helper.saveTicket(FOLDERNAME, ticket, evolutionStep, JIRA, sample_id)

The JSON was successfully saved!


# Annotate Ticket

In [ ]:
def annotateTicket(ticket, annotation, reason):
    try:
        ticket['ViolationActual'] = annotation
        ticket['ViolationReason'] = reason
        print("Annotation successful.")
    except:
        print("Annotation failed.")

annotateTicket(ticket, "TRUE", """1. Resolution has to be set from 'None' to 'Low Priority'.
                           2. Status has to be set from 'Open' to 'Resolved'.""")

In [ ]:
if os.path.isfile("data/" + FOLDERNAME + "/" + FOLDERNAME + "Dataset.csv"):
    dataset = pd.read_csv("./data/" + FOLDERNAME + "/" + FOLDERNAME + "Dataset.csv")
    print("The dataset already exists.")
else:
    dataset = pd.DataFrame(columns=['Jira', 'IssueId', 'EvoId', 'Summary', 'Description', 'VersionsAffected', 'IssueType', 'Project', 'Components', 'CreatedDate', 'ResolvedDate', 'Status', 'Priority', 'Creator', 'Reporter', 'Resolution', 'IssueLinks', 'Labels','VersionsFixed', 'Assignee', 'TimeSpent', 'Comments', 'ViolationActual', 'ViolationReason', 'FieldCount', 'Fields'])
    print("The dataset was created successfully.")

dataset = pd.concat([dataset, ticket], ignore_index=True)

dataset.to_csv("data/" + FOLDERNAME + "/" + FOLDERNAME + "Dataset.csv", index=False)
print("The ticket was inserted into the dataset successfully!")


In [ ]:
dataset = pd.read_csv('data/' + FOLDERNAME + '/' + FOLDERNAME + 'Dataset.csv')
dataset